In [38]:
import numpy as np
import cv2
import textwrap
import operator
import functools
from fitbert import FitBert
import pytesseract

In [39]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [40]:
img = cv2.imread('1.jpg', 0)
text = pytesseract.image_to_string(img)

In [41]:
print(text)

Name:

Nouns
Directions: Circle the words in the box that are nouns. Then, use those nouns to
complete the sentences below.

book run turtle globe pig

ball sing song jump pencil

table happy sip hand house

 

1. lused the to draw a picture for my mom.
2. Every night, we read a together in bed.
3. The dirty, little played in the mud puddle.

4. He put the pumpkin pie onto the kitchen.

5. We used the in school today to find Africa.

6. Sean cut his on the sharp edge of the desk.

7. On the PBS animal show, a was rescued.

8. My kitty likes to play with his new, rubber

9. It was my turn to pick the for our piano teacher.

10. Dad wants to paint the blue, but mom doesn’t.

©www.HaveFunTeaching.com



In [42]:
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
_, binary = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)
contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
idx = 0
mask = np.zeros_like(img)
cv2.drawContours(mask, sorted(contours, key=cv2.contourArea, reverse=True), idx, 255, -1)
out = np.zeros_like(img)
out[mask == 255] = img[mask == 255]


text = pytesseract.image_to_string(out)

In [43]:
def format_text(details):
    parse_text = []
    word_list = []
    last_word = ''
    for word in details['text']:
        if word != '':
            word_list.append(word)
            last_word = word
        if (last_word != '' and word == '') or (word == details['text'][-1]):
            parse_text.append(word_list)
            word_list = []

    return parse_text

In [44]:
def parse_text(threshold_img):
    tesseract_config = r'--oem 3 --psm 6'
    details = pytesseract.image_to_data(threshold_img, output_type=pytesseract.Output.DICT,
                                        config=tesseract_config, lang='eng')
    return details

In [45]:
parsed_data = parse_text(out)
arranged_text = format_text(parsed_data)
options = functools.reduce(operator.iconcat, arranged_text, [])

In [46]:
print(options)

['book', 'run', 'turtle', 'globe', 'pig', 'ball', 'sing', 'song', 'jump', 'pencil', 'table', 'happy', 'sip', 'hand', 'house']


In [47]:
fb = FitBert()

device: cuda
using model: bert-large-uncased


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
unmasked_string = "We used the in school today to find Africa."
span_to_mask = (17, 22)
masked_string, masked = fb.mask(unmasked_string, span_to_mask)

filled_in = fb.fitb(masked_string, options=options)

In [49]:
print(filled_in)

We used the in sc globe today to find Africa.
